# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [218]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
alt.data_transformers.disable_max_rows() #línea cambiada según instrucciones
alt.themes.enable('opaque')


%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [219]:
start_date = pd.to_datetime("2020-04-13")

In [264]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
0,247552.0,270.0,2020-04-27,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,109.067994
1,247552.0,9138.0,2020-10-12,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,3691.345657
2,247552.0,596.0,2020-05-29,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,240.757497
3,247552.0,6131.0,2020-08-17,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,2476.651370
4,247552.0,353.0,2020-05-11,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,142.596303


In [289]:
confirmados[confirmados.loc[:,"comuna"]=="Arica"].sort_values("fecha") #se confirma que los valores van ascendiendo, es decir, son acumulados. 
#Se revisó la base de datos pero se debía comprobar

,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
31,247552.0,115.0,2020-04-13,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,46.454886
51,247552.0,124.0,2020-04-15,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,50.090486
28,247552.0,134.0,2020-04-17,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,54.130041
38,247552.0,166.0,2020-04-20,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,67.056618
39,247552.0,224.0,2020-04-24,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,90.486039
...,...,...,...,...,...,...,...,...,...,...
48,247552.0,10294.0,2020-11-13,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,4158.318252
9,247552.0,10363.0,2020-11-16,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,4186.191184
67,247552.0,10443.0,2020-11-20,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,4218.507627
23,247552.0,10489.0,2020-11-23,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,4237.089581


In [223]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0
4,Tarapaca,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0


In [224]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

,region,fecha,fallecidos
374,Arica y Parinacota,2020-04-13,1.0
375,Tarapacá,2020-04-13,0.0
376,Antofagasta,2020-04-13,1.0
377,Atacama,2020-04-13,0.0
378,Coquimbo,2020-04-13,0.0


In [225]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.head()

,grupo_de_edad,fecha,fallecidos
28,0-39,2020-04-13,3
29,40-49,2020-04-13,2
30,50-59,2020-04-13,3
31,60-69,2020-04-13,12
32,70-79,2020-04-13,29


## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos la fecha por cada grupo etáreo.

In [226]:
fallecidos1=alt.Chart(fallecidos_etareo).mark_line().encode(
    x=alt.X('fecha', axis=alt.Axis(title='Fecha')),
    y=alt.Y('fallecidos', axis=alt.Axis(title='Fallecidos')),
    color=alt.Color('grupo_de_edad', scale=alt.Scale(scheme='dark2'), legend=alt.Legend(title="Grupo de edad")),
    tooltip=[alt.Tooltip("grupo_de_edad",title="Grupo de edad")]).properties(title="Fallecidos vs Grupo Etario").interactive()

fallecidos1

alt.Chart(...)

**Comentarios:** Se nota una tendencia de que hay una mucho mayor cantidad de fallecidos en los grupos etarios mayores (sobre 60 años se nota marcadamente el cambio), a excepción del grupo sobre 90 años, pero para concluir algo sobre este grupo, se debe revisar la cantidad de habitantes pertenecientes a esta categoría.

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [301]:
act_norm=activos.groupby("comuna")["poblacion"].apply(lambda df: df.std()) #se calcula la desviación estándar de cada comuna para comprobar 
#de manera más evidente cuál es la variación de la población

vpoblacion_lineas=alt.Chart(activos).mark_rect().encode(
    x=alt.X('fecha:O', axis=alt.Axis(title="Fecha")),
    y=alt.Y('poblacion:Q', axis=alt.Axis(title='Población')),
    color=alt.Color('comuna:N', scale=alt.Scale(scheme="dark2"), legend=alt.Legend(title="Comunas")),
    tooltip=['comuna']).properties(title="Población por comunas por fecha",
    width=500, 
    height=500).interactive()
vpoblacion_barras=alt.Chart(activos).mark_rect().encode(   #se hace este segundo gráfico con los valores de la desviación estándar para comprobar la ausencia de cambios
    x=alt.X('fecha:O', axis=alt.Axis(title="Fecha")),
    y=alt.Y('act_norm:Q', axis=alt.Axis(title='Desviacion estándar población')),
    color=alt.Color('comuna:N', scale=alt.Scale(scheme="dark2"), legend=alt.Legend(title="Comunas")),
    tooltip= [alt.Tooltip("comuna",title="Comuna")]).properties(title="Población por comunas por fecha",
    width=500, 
    height=500).interactive()
vpoblacion_lineas & vpoblacion_barras

alt.VConcatChart(...)

**Comentarios:** No se presentan variaciones. Esto se puede ver ya que se está utilizando la base de datos del INE, generada desde el censo del 2017, por lo que no se tenía en consideración una pandemia que disminuye en cierto porcentaje la cantidad de habitantes, y se debe observar detalladamente la metodología de proyección, que probablemente modifica anualmente la cantidad de habitantes por comuna. 
Elegí usar este gráfico porque se adecúa a la única variable que está siendo evaluada que es la población a lo largo del tiempo por cada comuna. Sin embargo, en un gráfico de líneas se veía también que no había niguna variación. 

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [228]:
fallecidos_et=alt.Chart(fallecidos_etareo).mark_rect().encode(
    x=alt.X('fecha:O', axis=alt.Axis(title="Fecha")),
    y=alt.Y('fallecidos:Q', axis=alt.Axis(title='Fallecidos')),
    color=alt.Color('grupo_de_edad:N', scale=alt.Scale(scheme="dark2"), legend=alt.Legend(title="Grupo etáreo")),
    tooltip=[alt.Tooltip('fallecidos',title="Fallecidos"), alt.Tooltip("fecha",title="Fecha")]).properties(title="Fallecidos por grupo etáreo",
    width=600, 
    height=600).interactive()
fallecidos_et

alt.Chart(...)

**Comentarios:** Para este gráfico resulta más cómodo usar el mark_rect, ya que se pueden comparar las categorías fácilmente. Se puede ver en el gráfico que en el mes de Julio hubo un peak desde el cual, la tendencia ha ido en aumento hasta la fecha para todas las categorías, aunque en una muy baja tasa para los grupos etáreos iguales o inferiores a 49 años, por lo que no aparecen en el gráfico.

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [278]:
casos_conf2=alt.Chart(confirmados).mark_line().transform_aggregate(
    casos_confirmados="sum(casos_confirmados)", groupby=["fecha"]).encode(
    x=alt.X('fecha:O', axis=alt.Axis(title="Fecha")),
    y=alt.Y('casos_confirmados:Q', axis=alt.Axis(title='Casos Confirmados')),
    tooltip=[alt.Tooltip('casos_confirmados',title="Confirmados"), alt.Tooltip("fecha",title="Fecha")]).properties(title="Casos confirmados",
    width=300, 
    height=300).interactive()

fallecidos_tot=alt.Chart(fallecidos).mark_line().transform_aggregate(
    fallecidos="sum(fallecidos)", groupby=["fecha"]).encode(
    x=alt.X('fecha:O', axis=alt.Axis(title="Fecha")),
    y=alt.Y('fallecidos:Q', axis=alt.Axis(title='Fallecidos')),
    tooltip=[alt.Tooltip('fallecidos',title="Fallecidos"), alt.Tooltip("fecha",title="Fecha")]
).properties(title="Fallecidos",
    width=300, 
    height=300).interactive()

casos_act=alt.Chart(activos).mark_line().transform_aggregate(
    casos_activos="sum(casos_activos)", groupby=["fecha"]).encode(
    x=alt.X('fecha:O', axis=alt.Axis(title="Fecha")),
    y=alt.Y('casos_activos:Q', axis=alt.Axis(title='Casos Activos')),
    tooltip=[alt.Tooltip('casos_activos',title="Casos Activos"), alt.Tooltip("fecha",title="Fecha")]
).properties(title="Casos activos",
    width=300, 
    height=300).interactive()

casos_conf2&fallecidos_tot&casos_act

alt.VConcatChart(...)

**Comentarios:** Se puede ver como los casos activos fueron a la alza hasta el mes de Junio, hasta llegar a un peak en este mismo mes. Como consecuencia, hubieron muchos fallecidos durante el mes de Julio, que es cuando llegó este gráfico a un peak en cuanto a la tasa de fallecidos, por lo que se puede pensar que hubo un periodo de tiempo entre la detección de los casos activos y el fallecimiento de parte de esos enfermos, que pudo incluir tiempo de hospitalización de por medio. El gráfico de confirmados se comporta de igual manera al de fallecidos, ya contempla los casos confirmados acumulados. En cambio, el gráfico de casos activos no contempla a los fallecidos, sino que solo a los pacientes vivos y que hayan sido notificados hace 11 días o menos los inicios de sus síntomas, por lo que este gráfico varía de importante manera según las políticas restrictivas y los fallecidos de este virus. 

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [300]:
tasa_lineas=alt.Chart(confirmados).mark_line().transform_aggregate(
    tasa="sum(tasa)",groupby=["region","fecha"]).encode(
    x=alt.X('fecha:O', axis=alt.Axis(title="Fecha")),
    y=alt.Y('tasa:Q', axis=alt.Axis(title='Tasa')),
    color=alt.Color('region:N', scale=alt.Scale(scheme="dark2"), legend=alt.Legend(title="Comunas")),
    tooltip=[alt.Tooltip("region",title="Region"), alt.Tooltip('tasa',title="Tasa"), alt.Tooltip('fecha',title="Fecha")]
).properties(title="Tasa de Incidencia Regional Acumulada vs Tiempo",
    width=500, 
    height=500).interactive()
tasa_lineas

alt.Chart(...)

**Comentarios:** Se nota una tasa de incidencia acumulada totalmente desproporcionada en la Región Metropolitana, especialmente entre mediados de Mayo y mediados de Julio aproximadamente, al mismo tiempo en que se comenzaron a aplicar las medidas más estrictas de cuarentena a lo largo de Santiago y todo Chile. Esta situación se puede deber a la cantidad de habitantes y al hacinamiento en los medios de transporte públicos, aumentando el riesgo de contagio enormemente (Tomando en cuenta la definición de la tasa del dataset que cito al final de este bloque de comentario), y considerando un leve desfase entre la detección de los datos y el momento en que las personas estaban enfermas, ya que el comienzo de la cuarentena en Santiago comenzó igualmente a mediados de Mayo. 


"Se entiende por tasa de incidencia acumulada el número total de casos acumulados en relación a la población susceptible de enfermar en un período determinado".

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [313]:
confirmados13=confirmados[confirmados.loc[:,"fecha"]=="2020-04-13"]
confirmados6=confirmados[confirmados.loc[:,"fecha"]=="2020-11-06"]

scatter_conf=alt.Chart(confirmados13).mark_point().encode(
    x=alt.X('casos_confirmados:Q', axis=alt.Axis(title="Casos Confirmados")),
    y=alt.Y('tasa:Q', axis=alt.Axis(title='Tasa de Incidencia Ac.')),
    color=alt.Color('region:N', scale=alt.Scale(scheme="dark2"), legend=alt.Legend(title="Regiones")),
    tooltip=[alt.Tooltip("comuna",title="Comuna"), alt.Tooltip('tasa',title="Tasa"), alt.Tooltip('casos_confirmados',title="Casos Confirmados")],
    size=alt.Size('poblacion:Q', title='Poblacion')
).properties(title="Tasa Acc. vs Casos Confirmados 13 Abril",
    width=500, 
    height=500).interactive()


scatter_tasa=alt.Chart(confirmados6).mark_point().encode(
    x=alt.X('casos_confirmados:Q', axis=alt.Axis(title="Casos Confirmados")),
    y=alt.Y('tasa:Q', axis=alt.Axis(title='Tasa de Incidencia Ac.')),
    color=alt.Color('region:N', scale=alt.Scale(scheme="dark2"), legend=alt.Legend(title="Regiones")),
    tooltip=[alt.Tooltip("comuna",title="Comuna"), alt.Tooltip('tasa',title="Tasa"), alt.Tooltip('casos_confirmados',title="Casos Confirmados")],
    size=alt.Size('poblacion:Q', title='Poblacion'),
).properties(title="Tasa Acc. vs Casos Confirmados 6 Noviembre",
    width=500, 
    height=500).interactive()


scatter_conf & scatter_tasa

alt.VConcatChart(...)

**Comentarios:** Se nota una importante diferencia entre las tasas de incidencia acumuladas para los mismos casos de confirmados (rango 1-471), donde la tasa máxima es de Puente Alto de 272 para 387 casos confirmados, mientras que para la segunda fecha y una cantidad muy cercana de casos, de 381, se tiene un valor de la tasa de 3651 para la comuna de San Pedro de Atacama, lo cual significa que mucha gente se enfermó durante este período de tiempo, haciendo aumentar la tasa ya que es una tasa acumulada de incidencia para cada comuna, e implicando que algunas comunas multiplicaron sus tasas por al menos 10 veces su valor inicial para la misma cantidad de casos confirmados evaluados inicialmente. Por otro lado, los casos confirmados cambian entre ambas fechas desde un orden de magnitud de centenas en el eje de las abscisas, que alcanza hasta 470 casos solamente en la primera fecha, mientras que en la segunda fecha los casos llega hasta el orden de la decena de miles de casos confirmados, donde bastantes comunas se posicionaron.

Respecto a utilizar el tamaño de la población para modificar el diametro de las circunferencias, creo que es útil, ya que añade una dimensión a considerar a los gráficos, e incluso se puede notar cómo las circunferencias que representan a las comunas que poseen mayor población se posicionan más hacia la derecha, indicando una mayor cantidad de casos confirmados en ambos gráficos, y en el segundo denotando un notorio aumento en la tasa respecto a los casos igualmente. 

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [328]:
casos_act_comuna=alt.Chart(activos).mark_rect().encode(
    x=alt.X('fecha:O', axis=alt.Axis(title="Fecha")),
    y=alt.Y('casos_activos:Q', axis=alt.Axis(title='Casos Activos')),
    color=alt.Color('comuna:N', scale=alt.Scale(scheme="dark2"), legend=alt.Legend(title="Comunas")),
    tooltip=[alt.Tooltip("casos_activos",title="Casos Activos"), alt.Tooltip('comuna',title="Comuna"), alt.Tooltip('region',title="Region"), 
    alt.Tooltip('fecha',title="Fecha")],
    size=alt.Size('poblacion:Q', title='Poblacion')
).properties(title="Casos Activos vs Tiempo",
    width=500, 
    height=500).interactive()
casos_act_comuna

alt.Chart(...)

In [357]:
graficos=[]
cont=0
for i in activos["region"].unique():
        cont+=1
        regioni=activos[activos.loc[:,"region"]==str(i)]
        casos_act_comuna_region=alt.Chart(regioni).mark_rect().encode(
        x=alt.X('fecha:O', axis=alt.Axis(title="Fecha")),
        y=alt.Y('casos_activos:Q', axis=alt.Axis(title='Casos Activos')),
        color=alt.Color('comuna:N', scale=alt.Scale(scheme="dark2"), legend=alt.Legend(title="Comunas")),
        tooltip=[alt.Tooltip("casos_activos",title="Casos Activos"), alt.Tooltip('comuna',title="Comuna"), alt.Tooltip('fecha',title="Fecha")],
        size=alt.Size('poblacion:Q', title='Poblacion')
    ).properties(title="Casos Activos vs Tiempo Region "+ str(i)+" ("+str(cont)+")", #se agrega un número al título
        width=500,                                                                #para facilitar identificación de la cantidad de gráficos
        height=500).interactive()
        graficos.append(casos_act_comuna_region)

alt.vconcat(*graficos)

alt.VConcatChart(...)

## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [15]:
region_names = {
    "Araucanía": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
}

In [16]:
# FIX ME #

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [5]:
import geopandas as gpd
from pathlib import Path

ModuleNotFoundError: No module named 'geopandas'

In [18]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

,Region,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,geometry
0,Región Metropolitana de Santiago,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,"POLYGON ((-7873736.745 -3885505.642, -7873695...."
1,Región de Antofagasta,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,Región de Arica y Parinacota,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,"POLYGON ((-7727277.278 -1997230.768, -7726464...."
3,Región de Atacama,1089,4,3,75661.248635,9.643906e+10,2.401741e+06,"MULTIPOLYGON (((-7900342.628 -3153340.296, -79..."
4,Región de Aysén del Gral.Ibañez del Campo,1088,14,11,106703.377369,2.242743e+11,4.144481e+07,"MULTIPOLYGON (((-8208500.834 -5733817.475, -82..."


In [19]:
type(regiones)

geopandas.geodataframe.GeoDataFrame

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [ ]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [ ]:
casos_geo = (
    # FIX ME #
)

casos_geo.head()

Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [ ]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [ ]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = # FIX ME #
    
    chart = alt.Chart(data).mark_geoshape().encode(
        # FIX ME #
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [ ]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive, interact

In [ ]:
col_widget = # FIX ME #

In [ ]:
fecha_widget = # FIX ME #

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [ ]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

**Comentarios:** RESPONDE AQUÍ